In [1]:
# Initialization

In [2]:
# References

# https://www.geeksforgeeks.org/priority-queue-in-python/
# https://docs.python.org/3/tutorial/classes.html
# https://www.geeksforgeeks.org/queue-in-python/
# https://www.programiz.com/dsa/priority-queue


In [3]:
# import libraries
import pandas as pd
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot as plt
import re


In [4]:
import copy

In [5]:
# read in data frames
full_list = pd.read_csv("cleaned_lax_list.csv")
smiley_list = pd.read_csv("Cleaned_Smiley_List_Lax.csv") #.dropna()

# can constrain the amount of words for efficient testing purposes

#full_list = full_list[:100]
#smiley_list = smiley_list[:100]

In [44]:
full_list

,cleaned_ID,ID,Name,URAP_id,URAP_label,Alt_name,First_chap,First_page,Gender,Unnamed: 8
0,7.0,7.0,Aðalsteinn inn sigrsæli Játvarðs son,431.0,King Athelstan of England,foster father of King Hakon,9.0,284.0,1.0,NaN
1,42.0,43.0,Aldís Ljótsdóttir (Hólmgöngu-Ljóts),613.0,Aldis,seduced by Ospak,50.0,374.0,0.0,NaN
2,51.0,53.0,Álfdís Konálsdóttir,404.0,Alfdis,Olaf Feilan's wife,7.0,281.0,0.0,NaN
3,68.0,70.0,Álfr Eysteinsson,390.0,Alf of Dalir,husband of Halldis,6.0,280.0,1.0,NaN
4,109.0,111.0,Án hrísmagi eða inn svarti,483.0,An the Black,"An Twig-belly, member of Olaf's household",24.0,315.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
341,6743.0,7171.0,Þórvör Ormsdóttir,672.0,Thorvor,daughter of Herdis and Orm,78.0,420.0,0.0,NaN
342,6793.0,7222.0,Þuríðr Ásgeirsdóttir ins yngra,590.0,Thurid,daughter of Asgeir Sr.,40.0,345.0,0.0,NaN
343,6800.0,7229.0,Þuríðr Eyvindardóttir Austmanns,360.0,Thurid,"daughter of Eyvind, wife of Thorstein, sister ...",4.0,278.0,0.0,NaN
344,6814.0,7244.0,Þuríðr Höskuldsdóttir,438.0,Thurid,daughter of Hoskuld and Jorunn,9.0,285.0,0.0,NaN


In [45]:
smiley_list

,Smiley_ID,Smiley_Name
0,5,Aldis Ljotsdottir (the Dueller)
1,7,Alf of Dalir
2,10,Alfdis (wife of Olaf Thorsteinsson Feilan)
3,17,An the Black/Twig-belly
4,18,An the White
...,...,...
345,1151,Vifil (a bondsman of Aud/Unn the Deep-minded. ...
346,1153,Vigdis Hallsteinsdottir (wife of Killer-Hrapp)
347,1154,Vigdis Ingjaldsdottir (wife of Thord Goddi)
348,1156,Vigdis Thorsteinsdottir (the Red)


In [6]:
# Basic class that contains the information of candidate words
# with two attributes: word id and levenshtein distance
class Candidate:
    # constructor
    def __init__(self, id, dis):
        self.id = id
        self.dis = dis
    # overload string method
    def __str__(self):
        return str([self.id, self.dis])
    # overload comparator methods in order for sort method to work
    def __lt__(self, other):
        return self.dis < other.dis
    def __gt__(self, other):
        return self.dis > other.dis
    def __eq__(self, other):
        return self.dis == other.dis
    # return lev distance
    def getDis(self):
        return self.dis
    # return word ID
    def getID(self):
        return self.id
    # actually unnecessary and should be deleted; same as __gt__ method
    def isgreater(self, other):
        return self.dis > other.dis
    

In [7]:
# Class miniCand works as a priority queue that limits the number of elements
class miniCand:
    # Constructor; thres determines the max number of elements in a queue
    def __init__(self, thres):
        self.col = []
        self.thres = thres
        self.maxdis = 1000
        
    # Insert method appends the new element to the end of the queue
    # if 1. there are empty slots 2. when there is no empty slot, 
    # the new element with shorter distance is having shorter distance
    # than the element with longest distance in the queue
    def insert(self, cand):
        if len(self.col) >= self.thres:
            if cand.getDis() < self.maxdis:
                self.col = sorted(self.col)
                self.col.pop()
                self.col.append(cand)
                if (len(self.col) > 1):
                    self.maxdis = max(self.col[-1].getDis(), self.col[-2].getDis())
                else:
                    self.maxdis = self.col[-1].getDis()
            else:
                return
        else:
            self.col.append(cand)
            self.maxdis = min(self.maxdis, cand.getDis())
    # pop the element with shortest distance
    def pop(self):
        self.col = sorted(self.col)
        return self.col.pop(0)
    # check is the queue is empty
    def isEmpty(self):
        return len(self.col) == 0
    # delete the word with shortest distance
    # and return the list of other candidates
    def clean(self):
        self.col = sorted(self.col)
        rest = self.col[1:]
        temp = self.col.pop(0)
        self.col.clear()
        self.col.append(temp)
        self.maxdis = temp.getDis()
        return rest
    # return a list containing the ID and distances of the elements
    def show(self):
        return [(i.getID(), i.getDis()) for i in self.col]
    # return the ID of the word with shortest distance
    def review(self):
        return self.col[0].getID()
    # return the number of elements
    def length(self):
        return len(self.col)
        

In [8]:
# Note: the methods below regarding the levenshtein distance may not all be used; 
# Retaining them is simply for the purpose of possible future uses, such as testing on
# another implementation of Levenshtein distance

In [9]:
# Input: two strings (one from full list, one from smiley list)
# output: a float object showing the levenshtein distance between two strings
def levenshteinDistanceForSaga(string1, string2):
    # assign length of strings
    len_string1 = len(string1)
    len_string2 = len(string2)
    # initialize the distance matrix with all 0 values; zeros requires dtype as parameter i.e. (4, 8)
    distanceMatrix = np.zeros(((len_string1 + 1), (len_string2 + 1)))
    # initialize values for string 1: from 0 to len - 1 i.e distance between current substring and an empty string
    for i in range((len_string1+1)):
        distanceMatrix[i][0] = i
    for j in range((len_string2+1)):
        distanceMatrix[0][j] = j
    # check: return distanceMatrix
    
    # loop through each letter in both words pairwise to compare
    # like a recursion design but with naive implementation, need to account for memory allocation
    # if there happens to be case of large strings
    for k in range(1, len_string1 + 1):
        for p in range(1, len_string2 + 1):
            # if two letters are the same
            # assign the rightbottom value as the lefttop value
            
            # initialize three values on the other three blocks of the square
            lefttop = 0
            leftbot = 0
            righttop = 0
            if (string1[k-1] == string2[p-1]):
                distanceMatrix[k][p] = distanceMatrix[k-1][p-1]
            else:
                # assign three values 
                lefttop = distanceMatrix[k-1][p-1]
                leftbot = distanceMatrix[k][p-1]
                righttop = distanceMatrix[k-1][p]
                
                # pick the min among three values and assign current block i.e. [k][p] plus 1 as cost
                if ((lefttop <= leftbot) and (lefttop <= righttop)):
                    distanceMatrix[k][p] = lefttop + 1
                elif ((leftbot <= lefttop) and (leftbot <= righttop)):
                    distanceMatrix[k][p] = leftbot + 1
                else:
                    distanceMatrix[k][p] = righttop + 1
    # note that the right lower value is the distance between two words
    return distanceMatrix[len_string1][len_string2]

In [10]:
# Input: two strings
# output: levenshtein distance between the first word in two strings
def firstWordLev(string1, string2):
    cut_string1 = string1.split(" ")[0] # assume no extreme case
    cut_string2 = string2.split(" ")[0]
    return levenshteinDistanceForSaga(cut_string1, cut_string2)
    

In [11]:
# input: two strings
# output: mininum levenshtein distance between the first word in first string and any word in the second string
def anyWordLev(string1, string2):
    cut_string1 = string1.split(" ")[0] # assume no extreme case
    cut_stringSet2 = string2.split(" ") # include content in parentheses in order to avoid coincidental miss
    min_lev = levenshteinDistanceForSaga(cut_string1, cut_stringSet2[0])
    temp_lev = -1
    
    for i in cut_stringSet2[1:]:
        temp_lev = levenshteinDistanceForSaga(cut_string1, i)
        if (temp_lev < min_lev):
            min_lev = temp_lev
        
    if (min_lev <= 2):
        return True
    return False

In [12]:
rep_list = ['\\','`','*','_','{','}','[',']','(',')','>','#','+','-','.','!','$','\"', '|','~', '/', ';', ':']
rep_list2 = ['\\','`','*','_','{','}','[',']','>','#','+','-','.','!','$','\"', '|','~', '/', ';', ':', ',']
single_quote = '\''
identity_list = set()

In [13]:
def stringListProcessing(string1, record_deletion):
    modifier = 0
    len_string1 = len(string1) - 1
    for i in record_deletion.keys():
        typeDel = record_deletion[i][0]
        freq = record_deletion[i][1]
        # print(typeDel, freq)
        if i < len_string1:
            for o in range(freq):
                # print(i - modifier)
                string1.pop(i - modifier)
            modifier += freq
                
    return string1
            
            

In [14]:
def processThe(string1, i, record_deletion, identity_dic):
    len_string1 = len(string1)
    second_locker = False
    if i < (len_string1 - 1):
        j = i + 1
        if 'the' not in identity_dic.keys():
            identity_dic['the'] = [string1[j]]
        else:
            identity_dic['the'].append(string1[j])
        record_deletion[i] = ['the', 2]
        j += 1
        while (j < len_string1):
            if (string1[j] in ['from', 'of', 'the']) or ('\'s' in string1[j]):
                return j - 1
            if (string1[j]) == "and":
                second_locker = True
            elif (second_locker):
                identity_dic['the'].append(string1[j])
                second_locker = False
            record_deletion[i][1] += 1
            j += 1
    return j

In [15]:
def processQuotationMark(string1, i, record_deletion, identity_dic):
    second_locker = False
    len_string1 = len(string1)
    if i < (len_string1 - 1):
        j = i + 1
        identity_list.add(string1[j])
        identity_dic[string1[j]] =  [string1[i].removesuffix('\'s')]
        record_deletion[i] = ['Quot', 2]
        j += 1
        while (j < len_string1):
            if (string1[j] in ['from', 'of', 'the']) or ('\'s' in string1[j]):
                return j - 1
            if (string1[j]) == "and":
                second_locker = True
            elif (second_locker):
                identity_dic[string1[i + 1]].append(string1[j])
                second_locker = False
            record_deletion[i][1] += 1
            j += 1
        return j - 1
    else:
        record_deletion[i] = ['Quot', 1]
        return i

In [16]:
def processFrom(string1, i, record_deletion, identity_dic):
    len_string1 = len(string1)
    if i < (len_string1 - 1):
        j = i + 1
        identity_dic['from'] = [string1[j]]
        record_deletion[i] = ['from', 2]
        j += 1
        while (j < len_string1):
            if (string1[j] in ['the', 'of', 'from']) or ('\'s' in string1[j]):
                return j - 1
                
            identity_dic["from"].append(string1[j])
            record_deletion[i][1] += 1
            j += 1
    return j
        
        
    

In [17]:
def processOf(string1, i, record_deletion, identity_dic):
    #second_locker = False
    len_string1 = len(string1)
    if i < (len_string1 - 1):
        j = i + 1
        identity_dic['of'] = []
        if string1[j] != 'the':
            identity_dic['of'].append(string1[j])
        record_deletion[i] = ['of', 2]
        j += 1
        while (j < len_string1):
            if (string1[j] in ['from', 'the', 'of']) or ('\'s' in string1[j]):
                return j - 1
                
            elif (string1[j] != 'and'):
                identity_dic["of"].append(string1[j])
            record_deletion[i][1] += 1
            j += 1
    return j - 1

In [18]:
def processComments(string1_copy, identity_dic):
    relation_index = []
    #belonging
    temp = ''
    temp2 = ''
    modifier = 0
    string1 = string1_copy[:]
    for i in preposition_list:
        for j in range(len(string1)):
            if i == string1[j]:
                relation_index.append(Candidate(i, j))
    #for k in range(len(string1)):
     #   if '\'s' in string1[k]:
      #      relation_index.append('\'s', k)
    relation_index = sorted(relation_index)
    if (len(relation_index) == 0):
        identity_dic['is'] = string1[:]
        return
    while (len(relation_index) > 0):
        temp = relation_index.pop(0).getDis() - modifier
        if (len(relation_index) > 0):
            temp2 = relation_index[0].getDis() - modifier
        else:
            temp2 = len(string1)
        # if the preposition is the starting word
        if temp == 0:
            identity_dic[string1[0]] = string1[1:temp2]
            
        #    modifier += temp2
        else:
            # if not
            identity_dic[string1[temp - 1]] = " ".join(x for x in string1[(temp+1):temp2] if x != 'and')
        modifier += temp2 
        string1 = string1[temp2:]
        
    
        
    
    

In [19]:
preposition_list = ['of', 'above', 'across', 'against', 'along', 'among', 'around', 'at', 'before', 'behind', 'below', 'beneath', 'beside', 'between', 'by', 'down', 'from', 'in', 'into', 'near', 'off', 'on', 'to', 'toward', 'under', 'upon', 'with', 'within']

In [20]:
relation_list = ['father', 'landholder', 'king', 'brothers', 'slave', 'men', 'wife', 'farmer', 'sister', 'son', 'daughter', 'earl', 'hersir', 'also', 'brother']

In [21]:
# assume that any identity indication is within the string 
def stringCleaningFull(string1):
    if (pd.isna(string1)):
        return "", {}
    identity_dic = {}
    record_deletion = {}
    string1 = string1.lower()
    for j in rep_list:
        if j in string1:
            if j == '-':
                string1 = string1.replace(j, ' ')
            else:
                string1 = string1.replace(j, '')
    string1 = string1.split(" ")
    string1_copy = string1[:]
    len_string1 = len(string1_copy)
    second_locker = False
    #print(len_string1)
    i = 0
    while i < len(string1_copy):
        if '\'s' in string1_copy[i]:
            i = processQuotationMark(string1, i, record_deletion, identity_dic) 
        elif 'from' == string1_copy[i]:
            i = processFrom(string1, i, record_deletion, identity_dic)
        elif 'of' == string1_copy[i]:
            i = processOf(string1, i, record_deletion, identity_dic)
        elif 'the' == string1_copy[i]:
            i = processThe(string1, i, record_deletion, identity_dic)
        i += 1

    if len(record_deletion) > 0:
        string1 = stringListProcessing(string1, record_deletion)
            
    return string1, identity_dic
            
    

In [22]:
# assume that any identity indication is within the string 
def stringCleaningSmiley(string1):
    # check is the name value is null
    if (pd.isna(string1)):
        return "", {}
    # record of identity phrases
    identity_dic = {}
    # index of all words that require deletion
    record_deletion = {}
    # turn to lower cases and remove whitespaces at front and back
    string1 = string1.lower().strip()
    
    option_parenthesis = False
    string_dic = ""
    identity_dic_supplement = {}
    strings_in_parenthesis = []
    
    # remove all marks
    for j in rep_list2:
        if j in string1:
            if j == '-' or j == '/':
                string1 = string1.replace(j, ' ')
            else:
                string1 = string1.replace(j, '')
    # print(string1)
    string1 = string1.split("(")
    # print(string1)
    for i in range(len(string1)):
        if ")" in string1[i]:
            index_of_paren = string1[i].index(")")
            strings_in_parenthesis.append(string1[i][:index_of_paren].strip())
            string1[i] = string1[i][index_of_paren:].replace(")", "")
    string1 = "".join(x for x in string1)
    if len(strings_in_parenthesis) >= 0: # more than 1 parenthesized sentences
        option_parenthesis = True
    string1_save = string1.split(" ")
    string1_copy = string1_save[:]
    #print(string1_copy)
    len_string1 = len(string1_copy)
    second_locker = False
    #print(len_string1)
    i = 0
    while i < len(string1_copy):
        if '\'s' in string1_copy[i]:
            i = processQuotationMark(string1_save, i, record_deletion, identity_dic) 
        elif 'from' == string1_copy[i]:
            i = processFrom(string1_save, i, record_deletion, identity_dic)
        elif 'of' == string1_copy[i]:
            i = processOf(string1_save, i, record_deletion, identity_dic)
        elif 'the' == string1_copy[i]:
            i = processThe(string1_save, i, record_deletion, identity_dic)
        i += 1

    if len(record_deletion) > 0:
        string1_save = stringListProcessing(string1_save, record_deletion)
    while '' in string1_save:
        string1_save.remove('')
    if (option_parenthesis):
        for i in range(len(strings_in_parenthesis)):
            string_dic = strings_in_parenthesis[i].strip().split(" ")
            processComments(string_dic, identity_dic_supplement)
        for j in identity_dic_supplement.keys():
            if j in identity_dic.keys():
                identity_dic[j]+= identity_dic_supplement[j]
            else:
                identity_dic[j] = identity_dic_supplement[j][:]

        return string1_save, identity_dic
            
    else:
        return string1_save, identity_dic
            

In [23]:
# input: two strings
# output: levenshtein distance between two names after modification in strings. Less constrains and is used for complex words
def desperateMatching(string1, string2):
    pattern_remove = r" \(.*\)"
    # turn to lower can 
    cut_stringSet1 = re.sub(pattern_remove, "", string1).split(" ")
    cut_stringSet2 = re.sub(pattern_remove, "", string2).split(" ")
    #print(cut_stringSet1, cut_stringSet2)
    while (len(cut_stringSet1) < len(cut_stringSet2)):
        cut_stringSet1 += [""]
    while (len(cut_stringSet1) > len(cut_stringSet2)):
        cut_stringSet2 += [""]
    min_index = 0
    min_lav = -1
    temp_lav = -1
    total_lav = 0
    while (len(cut_stringSet1) > 0):
        min_index = 0
        temp_lav = -1
        check_first = cut_stringSet1[0]
        min_lav = levenshteinDistanceForSaga(check_first, cut_stringSet2[0])
        for j in cut_stringSet2[1:]:
            temp_lav = levenshteinDistanceForSaga(check_first, j)
            if (temp_lav < min_lav):
                min_lav = temp_lav
                min_index = cut_stringSet2[1:].index(j) + 1
        total_lav += min_lav
        cut_stringSet2.remove(cut_stringSet2[min_index])
        cut_stringSet1.remove(cut_stringSet1[0])
        
    return total_lav
    

In [24]:
# input: two strings
# output: levenshtein distance between two names after modification in strings. Less constrains and is used for complex words
def desperateMatchingNew(string1, string2, dictionary1, dictionary2):
    cut_stringSet1 = string1[:]
    cut_stringSet2 = string2[:]
    # dictionary1 = copy.deepcopy(dictionary1_pre)
    # dictionary2 = copy.deepcopy(dictionary2_pre)
    while (len(cut_stringSet1) < len(cut_stringSet2)):
        cut_stringSet1 += [""]
    while (len(cut_stringSet1) > len(cut_stringSet2)):
        cut_stringSet2 += [""]
    min_index = 0
    min_lav = -1
    temp_lav = -1
    total_lav = 0
    penalty_decision = True
    while (len(cut_stringSet1) > 0):
        min_index = 0
        temp_lav = -1
        check_first = cut_stringSet1[0]
        min_lav = levenshteinDistanceForSaga(check_first, cut_stringSet2[0])
        if check_first[:2] == cut_stringSet2[:2]:
            min_lav -= 1
        for j in cut_stringSet2[1:]:
            temp_lav = levenshteinDistanceForSaga(check_first, j)
            if check_first[:2] == j[:2]:
                temp_lav -= 1
            if (temp_lav < min_lav):
                min_lav = temp_lav
                min_index = cut_stringSet2[1:].index(j) + 1
        total_lav += min_lav
        cut_stringSet2.remove(cut_stringSet2[min_index])
        cut_stringSet1.remove(cut_stringSet1[0])
    #total_lav += 5
    if (len(dictionary1) != 0) and (len(dictionary2) != 0):
        #total_lav -= 5
        for i in dictionary1.keys():
            for k in dictionary2.keys():
                if ((i == k) or ((i in ["from", "of", "by", "in", "at"]) and (k in ["from", "of", "by", "in", "at"]))):
                    #total_lav -= 1
                    for j in dictionary1[i]:
                        for l in dictionary2[k]:
                            if levenshteinDistanceForSaga(j, l) <= 3:
                                total_lav -= 3
                                penalty_decision = False
                                break
        if penalty_decision:
            total_lav += 2
    elif (len(dictionary1) == 0) or (len(dictionary2) == 0):
        if total_lav >= 3:
            total_lav += 3
        else:
            total_lav += 1
            
    # if total_lav < 0:
    #  total_lav = 0
        
        
    return total_lav

In [25]:
# input: two strings
# output: levenshtein distance between two names after modification in strings. More constrains and is used for first round
def moderateMatching(string1, string2):
    pattern_remove = r" \(.*\)"
    cut_stringSet1 = re.sub(pattern_remove, "", string1).split(" ")
    cut_stringSet2 = re.sub(pattern_remove, "", string2).split(" ")
    while (len(cut_stringSet1) < len(cut_stringSet2)):
        cut_stringSet1 += [""]
    while (len(cut_stringSet1) > len(cut_stringSet2)):
        cut_stringSet2 += [""]
    min_index = 0
    min_lav = -1
    temp_lav = -1
    total_lav = 0
    min_lav = levenshteinDistanceForSaga(cut_stringSet1[0], cut_stringSet2[0])
    total_lav += min_lav
    cut_stringSet2.remove(cut_stringSet2[0])
    cut_stringSet1.remove(cut_stringSet1[0])
    
    while (len(cut_stringSet1) > 0):
        min_index = 0
        temp_lav = -1
        check_first = cut_stringSet1[0]
        min_lav = levenshteinDistanceForSaga(check_first, cut_stringSet2[0])
        for j in cut_stringSet2[1:]:
            temp_lav = levenshteinDistanceForSaga(check_first, j)
            if (temp_lav < min_lav):
                min_lav = temp_lav
                min_index = cut_stringSet2[1:].index(j) + 1
        total_lav += min_lav
        cut_stringSet2.remove(cut_stringSet2[min_index])
        cut_stringSet1.remove(cut_stringSet1[0])
        
    return total_lav

In [26]:
# return a list of full List IDs that is sharing a smiley list word with 
# other full list IDs and is not the best match; update the corresponding lists regarding
# full List and Smiley List words accordingly. 
# Also, secure the best match for the smiley list words involved. 
def process(smileyID):
    smallestMatch = smileyList_Cand[smileyID].clean()
    smileyList_used[smileyID] = 1
    inCompleteIDs = [i.getID() for i in smallestMatch]
    for k in inCompleteIDs:
        if (not fullList_pref[k].isEmpty()):
            fullList_pref[k].pop()
        #fullList_ban[k].append(smileyID)
    return inCompleteIDs
    

In [27]:
# check is the matching is complete
def checkComplete():
    for i in fullList_confirm:
        if i != 1:
            return False
    return True

In [28]:
# the matching method after the first round of matching (for every word) is complete
def Normal_round(i):
    perm_name = full_list.loc[i, "URAP_label"]
    perm_name_string, perm_name_dic = stringCleaningSmiley(perm_name)
    fix_dis = 200
    fix_dis_2 = 200
    full_index = -1
    temp_name = ''
    change_id_mod = -1
    priority_value = -1
    final_round = True
    # if the preference list is not empty, use the current available and most preferred word
    # i.e. shortest distance as the match
    if (not fullList_pref[i].isEmpty()):
        final_round = False
        j = fullList_pref[i].pop().getID()
        while (smileyList_ban[j]):
            if (fullList_pref[i].isEmpty()):
                final_round = True
                break
            j = fullList_pref[i].pop().getID()
            
        temp_name = smiley_list.loc[j, "Smiley_Name"] 
        
        temp_name_string, temp_name_dic = stringCleaningSmiley(temp_name)
        if lavDisMatrix[j][i] == -1:
            lavDisMatrix[j][i] = desperateMatchingNew(perm_name_string, temp_name_string, perm_name_dic, temp_name_dic)
        temp_dis = lavDisMatrix[j][i]
        full_index = smiley_list.loc[j, "Smiley_ID"] 
    
        change_id_mod = j
        if (not final_round):
            smileyList_used[j] += 1
        smileyList_Cand[j].insert(Candidate(i, temp_dis))
        if (temp_dis <= 7):
            priority_value = 1
        elif (temp_dis <= 15):
            priority_value = 2
        else:
            priority_value = 3
    # if the preference list is empty, find the available smiley list word with shortest distance
    # and secure the match
    if final_round == True:
        for l in range(len(smiley_list)):
            if (smileyList_used[l] == 0):
                full_name = smiley_list.loc[l, "Smiley_Name"] 
                full_name_string, full_name_dic = stringCleaningSmiley(full_name) 
                if lavDisMatrix[l][i] == -1: 
                    lavDisMatrix[l][i] = desperateMatchingNew(perm_name_string, full_name_string, perm_name_dic, full_name_dic)        
                temp_dis = lavDisMatrix[l][i]
                #fullList_pref[i].insert(Candidate(l, temp_dis))
                if (temp_dis < fix_dis):
                    fix_dis = temp_dis
                    full_index = smiley_list.loc[l, "Smiley_ID"] 
                    temp_name = full_name
                    change_id_mod = l
        fullList_confirm[i] = 1
        smileyList_ban[change_id_mod] = True
        smileyList_used[change_id_mod] += 1
        smileyList_Cand[change_id_mod].insert(Candidate(i, fix_dis))
        if (temp_dis <= 7):
            priority_value = 1
        elif (temp_dis <= 15):
            priority_value = 3
        else:
            priority_value = 4
        

    Total_Single_dtFrame.loc[i] = [temp_name, full_index, 
                                   perm_name, full_list.loc[i, "URAP_id"], 
                                   temp_dis, full_list.loc[i, "Name"], 
                                   full_list.loc[i, "ID"], priority_value]    

In [29]:
# matching algorithm for the first round
def First_round(i):
    # make marks for incomplete rows
    if (type(full_list.loc[i, "URAP_label"]) == type(float('nan'))):
        Total_Single_dtFrame.loc[i] = ["", -1, "", -1, -1, "", -1, -1]
        return
    perm_name = full_list.loc[i, "URAP_label"]
    perm_name_string, perm_name_dic = stringCleaningSmiley(perm_name)
    fix_dis = 2000
    full_index = -1
    temp_name = ''
    change_id_mod = -1
    priority_value = -1 
    # find smiley word with shortest distance regardless of duplicated matches
    # and secure (or, make no long available) the smiley word having at least
    # a 0 lev distance matching 
    for j in range(len(smiley_list)):
        if smileyList_ban[j] == True:
            continue
        full_name = smiley_list.loc[j, "Smiley_Name"] 
        # moderateMatching
        
        full_name_string, full_name_dic = stringCleaningSmiley(full_name)
        if lavDisMatrix[j][i] == -1:   
            lavDisMatrix[j][i] = desperateMatchingNew(perm_name_string, full_name_string, perm_name_dic, full_name_dic)
        temp_dis = lavDisMatrix[j][i]
        fullList_pref[i].insert(Candidate(j, temp_dis))
        if (temp_dis < fix_dis):
            fix_dis = temp_dis
            if fix_dis == 0:
                smileyList_ban[j] = True
            full_index = smiley_list.loc[j, "Smiley_ID"] 
            temp_name = full_name
            change_id_mod = j
    smileyList_used[change_id_mod] += 1
    smileyList_Cand[change_id_mod].insert(Candidate(i, fix_dis))
    if fix_dis == 0:
        priority_value = 0
    elif (temp_dis <= 7):
            priority_value = 1
    elif (temp_dis <= 15):
            priority_value = 2
    else:
            priority_value = 3
    
    Total_Single_dtFrame.loc[i] = [temp_name, full_index, 
                                   perm_name, full_list.loc[i, "URAP_id"], 
                                   fix_dis, full_list.loc[i, "Name"], 
                                   full_list.loc[i, "ID"], priority_value] 
    

In [30]:
smiley_list

,Smiley_ID,Smiley_Name
0,5,Aldis Ljotsdottir (the Dueller)
1,7,Alf of Dalir
2,10,Alfdis (wife of Olaf Thorsteinsson Feilan)
3,17,An the Black/Twig-belly
4,18,An the White
...,...,...
345,1151,Vifil (a bondsman of Aud/Unn the Deep-minded....
346,1153,Vigdis Hallsteinsdottir (wife of Killer-Hrapp)
347,1154,Vigdis Ingjaldsdottir (wife of Thord Goddi)
348,1156,Vigdis Thorsteinsdottir (the Red)


In [31]:
# remove empty spaces
for i in range(len(smiley_list)):
    smiley_list.loc[i, "Smiley_Name"] = smiley_list.loc[i, "Smiley_Name"].strip()
for i in range(len(full_list)):
    if (type(full_list.loc[i, "URAP_label"]) != type(float('nan'))):
        full_list.loc[i, "URAP_label"] = full_list.loc[i, "URAP_label"].strip()

In [32]:
# length (number of rows) of dataframes
lenSmiley = len(smiley_list)
lenFull = len(full_list)

#candidate_list = []
# queue that contains the smiley word IDs in order for matching 
search_order = deque()
# the boolean variable indicating is the first round is complete
firstRoundIndicator = True
for i in range(lenFull):
 #   candidate_list.append(PQ())
    search_order.append(i)

In [33]:
# candidiate list for smiley words
smileyList_Cand = []
# number of uses of current smiley words
smileyList_used = []
# ban list for the smiley word i.e. which full list word can never be \
# used to match with this smiley word
smileyList_ban = []
#fullList_ban = []

# preference list of full list words in order of lev distances
# in future, more attributes and more complicated algorithms can be used 
# to determine the order in preference list, such as the probability of correct match
fullList_pref = []
# the confirmation of whether a full list word has the guaranteed match
fullList_confirm = []
# fill in the lists
for i in range(lenSmiley):
    smileyList_Cand.append(miniCand(lenFull))
    smileyList_used.append(0)
    smileyList_ban.append(False)
for j in range(lenFull):
    #fullList_ban.append(list())
    fullList_pref.append(miniCand(3))
    fullList_confirm.append(0)

In [34]:
# extract columns that indicates characters' belonging to sagas
list_cols = smiley_list.columns[4:]

# Initialize dictionaries, storing name ID as the value for names
full_name_index = {}
smiley_name_index = {}

# record name IDs (values) to names (indices) 
# full list
for i in range(len(full_list)):
    full_name_index[full_list.loc[i, "Name"]] = full_list.loc[i, "ID"]
# smiley list
for j in range(len(smiley_list)):
    smiley_name_index[smiley_list.loc[j, "Smiley_Name"]] = smiley_list.loc[j, "Smiley_ID"]

In [35]:
# Initialize outputting dataframe
Total_Single_dtFrame = pd.DataFrame(columns=["Smiley_Name", 'Smiley_ID', 'URAP_Name', 'URAP_ID', 'lav_dis', 'Full_Name', "Full_ID", "Priority"], index= range(len(full_list)))

In [36]:
# variables for testing purposes and for avoiding infinite loop
iteraing = 0
ring = 0 

In [37]:
# continue if 1. the searching queue is not empty 
# 2. some full list words need a (better or unique) match
lavDisMatrix = [[-1 for i in range(len(full_list))] for j in range(len(smiley_list))]

while ((len(search_order) > 0) or (not checkComplete())):
    # process the lists when the searching queue is empty
    if (len(search_order) == 0):
        # print("before cleaning", smileyList_used)
        firstRoundIndicator = False
        requiredList = set()
        # confirm matching with lev distance of 0 
        for k in range(lenFull):
            if (Total_Single_dtFrame.loc[k, "lav_dis"] <= 0):
                fullList_confirm[k] = 1
        # for duplicated uses of smiley list words, retain the best matches
        # and put other full list words into searching queue
        for i in range(lenSmiley):
            if (smileyList_used[i] > 1):
                requiredList.update(process(i))
        # print("after cleaning", smileyList_used)
        print(len(requiredList))
        for j in requiredList:
            search_order.append(j)
        # if there is no duplicate matches, we may know that the entire
        # matching process is done, and we may end
        if len(requiredList) == 0:
            for i in range(len(fullList_confirm)):
                fullList_confirm[i] = 1
        continue
    # if the searching queue is not empty:
    # pop and store the first full List word ID 
    i = search_order.popleft()
    # if it is not first round: use Normal_round function
    if (not firstRoundIndicator):
        Normal_round(i)
    else:
        # otherwise, use First_round function
        First_round(i)
    # if the matching is repeating more than the most possible rounds 
    # (depending on max size of preference list), 
    # end the searching process
    ring += 1
    if ring >= 3 * lenFull:
        break
    

218
212
0


In [38]:
# lavDisMatrix

In [39]:
# export modified full list
# apply_full_list.to_csv("SingleWord_Check_Full_List_Match.csv")
Total_Single_dtFrame

,Smiley_Name,Smiley_ID,URAP_Name,URAP_ID,lav_dis,Full_Name,Full_ID,Priority
0,Athelstan the Victorious/the Faithful,67,King Athelstan of England,431.0,6.0,Aðalsteinn inn sigrsæli Játvarðs son,7.0,3
1,Hjaltasons (Thord and Thorvald at Hof),437,Aldis,613.0,21.0,Aldís Ljótsdóttir (Hólmgöngu-Ljóts),43.0,4
2,Alfdis (wife of Olaf Thorsteinsson Feilan),10,Alfdis,404.0,1.0,Álfdís Konálsdóttir,53.0,3
3,Alf of Dalir,7,Alf of Dalir,390.0,-3.0,Álfr Eysteinsson,70.0,3
4,An the Black/Twig-belly,17,An the Black,483.0,-3.0,Án hrísmagi eða inn svarti,111.0,3
...,...,...,...,...,...,...,...,...
341,Arnbjorn Sleitu-Bjarnarson,31,Thorvor,672.0,26.0,Þórvör Ormsdóttir,7171.0,4
342,Turf-Einar Rognvaldsson,1112,Thurid,590.0,24.0,Þuríðr Ásgeirsdóttir ins yngra,7222.0,4
343,Vigdis Thorsteinsdottir (the Red),1156,Thurid,360.0,24.0,Þuríðr Eyvindardóttir Austmanns,7229.0,4
344,Osk Thorsteinsdottir (the Red),657,Thurid,438.0,25.0,Þuríðr Höskuldsdóttir,7244.0,4


In [43]:
output_available_txt = ""
for i in range(len(full_list)):
    output_available_txt += "NAME " + str(i) + ": "
    for j in range(len(smiley_list)):
        if URAP_page_dic[i] in smiley_page_matrix[int(smiley_list.loc[j, "Smiley_ID"]) - 1]:
            output_available_txt += str(smiley_list.loc[j, "Smiley_Name"]) + "|" + str(smiley_list.loc[j, "Smiley_ID"]) + "|" + ", "
    
    output_available_txt += "\n"

NameError: name 'URAP_page_dic' is not defined

In [40]:
output = ""
for i in range(lenSmiley):
    if smileyList_used[i] == 0:
        output += smiley_list.loc[i, "Smiley_Name"] + " " + str(smiley_list.loc[i, "Smiley_ID"]) + "\n"

In [41]:
# export the data frame into .csv file
Total_Single_dtFrame.to_csv("Laxdoela_Result2.csv")

In [42]:
# Writing into .txt file
text_file = open("Null Laxdoela List2.txt", "w")
text_file.write(output)
text_file.close()